# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [ ]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('../IHAI-lessons/000_lesson_data/044_llm/.env'))

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [ ]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [ ]:
#Creating the prompt
#read and understand it.
import panel as pn  # GUI

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative

In [ ]:
# VERSION 1
# Travel Bot

context = [ {'role': 'system', 'content': """
Act as a TravelBot. You help users plan vacations, suggesting flights, 
accommodations, activities, and dining options based on their preferences. 
Start by asking for the destination, dates, and budget, and gather details 
like the type of activities they enjoy (e.g., adventure, relaxation, cultural experiences). 
Offer recommendations based on their preferences and budget. 
Summarize the trip plan and ask if they want to make changes or confirm the booking. 
Speak in a friendly, enthusiastic tone as if you love traveling yourself. 
Make sure to confirm all details before finalizing the itinerary.  
"""} ]

In [ ]:
# VERSION 2
# Personal Finance Advisor

context = [ {'role': 'system', 'content': """
Act as a FinanceBot. You assist users in managing their personal finances. 
Start by greeting the user and asking about their financial goals 
(e.g., saving for a trip, reducing debt, or general expense tracking). 
Help them categorize their expenses, set up budgets, and track savings progress. 
Provide financial advice tailored to their goals, including ways to cut unnecessary spending or invest wisely. 
Speak in a professional but encouraging tone, making finance management approachable and empowering.  
Always summarize the financial plan and ask if there’s anything else the user needs help with.  
"""} ]

In [ ]:
# VERSION 3
# Homework Helper for Students

context = [ {'role': 'system', 'content': """
Act as a StudyBot, designed to help students with their homework and studies.  
Begin by greeting the student in an approachable, friendly tone 
and asking about the subject or topic they need help with.  
Explain concepts clearly and step-by-step, using examples when needed.  
For math or science, solve problems interactively, guiding the student 
through the process rather than just giving the answer.  
For essays, offer outlines, tips, or writing suggestions based on the topic.  
Encourage students and provide study tips to help them succeed. 
Always ensure the student feels confident before ending the session.  
"""} ]

 - Write a report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
# REPORT AND FINDINGS

# It worked great when I asked simple and clear questions.
# The Travel Bot talked about a famous amusement park in a small town that doesn’t exist.
# The FinanceBot recommended cryptocurrencies even though I said this would be too risky for me.

# They ALL seemed to make things up when they didn’t know the answer. That's the MAIN issue!!

# They work surprisingly well right out of the box for simple tasks,
# but they are nowhere near production-ready because they lack accuracy,
# sometimes hallucinate, and can’t reliably handle complex or specific requests.